In [1]:
from importlib import reload

import os
from rdkit import Chem
from copy import deepcopy
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

from serenityff.charge.tree.tree import tree
from serenityff.charge.tree.node import node
from serenityff.charge.tree_develop.tree_constructor import Tree_constructor
from serenityff.charge.tree.tree_utils import create_new_node_from_develop_node

/home/mlehner/.conda/envs/serenityff-charge-test/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#data_file="/localhome/mlehner/sff_data/tot_all.csv"
data_file="/localhome/mlehner/sff_data/sff_training_all_niels.csv"
sdf_suply="/localhome/mlehner/sff_data/combined.sdf"
nrows=10000000
#nrows=1000
data_split = 0.01
out_folder = "/home/mlehner/serenityff-charge/dev/test_jnbs/test_004_out"

In [3]:
tree_constructor = Tree_constructor(df_path=data_file, sdf_suplier=sdf_suply, nrows=nrows, num_layers_to_build=5, data_split=data_split, verbose=True)#, read_engine="pyarrow")

2022-09-02 15:42:02.912328	Initializing Tree_constructor
2022-09-02 15:42:02.912417	Mols imported, starting df import


In [ ]:
tree_constructor.create_tree_level_0()

In [ ]:
tree_constructor.build_tree()

In [ ]:
tree_constructor.root.children[0]

In [ ]:
tree_constructor.convert_tree_to_node(delDevelop=True)

In [ ]:
tree_constructor.new_root.children[0]

In [ ]:
tree_constructor.new_root

In [ ]:
tree_constructor.new_root.fix_nan_stdDeviation()

In [ ]:
for num, child in enumerate(tree_constructor.new_root.children):
    child.to_file(f"{out_folder}/tree/tree_{num}.csv")

In [ ]:
#tree_constructor.write_tree_to_file(out_folder+"/tree.csv")

In [ ]:
tree_constructor.test_df.to_csv(out_folder+"/test_df.csv")

In [ ]:
new_tree = tree()
#new_tree.from_folder(out_folder+"/tree", verbose=True)
new_tree.root = tree_constructor.new_root

In [ ]:
new_tree.root.children[1].children[0]

In [ ]:
for child in new_tree.root.children:
    child.prune()

In [ ]:
def try_to_merge_similar_branches(current_node, min_deviation=0.001, children_overlap_acceptance=0.6):
        """
        Try to merge similar branches.

        Parameters
        ----------
        min_deviation : float, optional
            threhold for similarity, by default 0.001
        children_overlap_acceptance : float, optional
            threshold for overlap similarity, by default 0.6
        """
        for idx, child in enumerate(current_node.children):
            for other_child in current_node.children[idx + 1 :]:
                if child.node_is_similar(other_child, min_deviation=min_deviation):
                    control_bool = True
                    child_match = 0
                    for node_i in child.children:
                        if node_i in other_child.children:
                            child_match += 1
                            if not node_i.node_is_similar(
                                other_child.children[other_child.children.index(node_i)],
                                min_deviation=min_deviation,
                            ):
                                control_bool = False
                                break
                    if (
                        control_bool
                        and len(child.children) > 0
                        and ((child_match / len(child.children)) >= children_overlap_acceptance)
                    ):
                        child.add_node(other_child)
                        current_node.children.remove(other_child)
            try_to_merge_similar_branches(child, min_deviation=min_deviation)


In [ ]:
try_to_merge_similar_branches(new_tree.root)

In [ ]:
new_tree.root.fix_nan_stdDeviation()

In [ ]:
#tree_constructor.new_root.to_file(out_folder+"/tree2.csv")

In [ ]:
for num, child in enumerate(new_tree.root.children):
    child.to_file(f"{out_folder}/tree_pruned/tree_{num}.csv")

In [ ]:
new_tree.root.children

In [ ]:
test_tree = tree()

In [ ]:
test_tree.root = new_tree.root

In [ ]:
data_external = "/localhome/mlehner/sff_data/external_test_set_0707.sdf"
data_solv = "/localhome/mlehner/sff_data/test_set_solf.sdf"

In [ ]:
test_tree.root.children[0]

In [ ]:
df_solv = test_tree.match_dataset(mol_sup=Chem.SDMolSupplier(data_solv, removeHs=False))

In [ ]:
df_solv.head(2)

In [ ]:
def calculate_RMSE_tree_vs_truth(df, arg1, arg2):
    return ((df[arg1]-df[arg2])**2).mean()**0.5

def calculate_r2_tree_vs_truth(df, arg1, arg2):
    return df[[arg1, arg2]].corr()[arg1][arg2]**2

In [ ]:
calculate_RMSE_tree_vs_truth(df_solv, "truth", "tree_norm2")

In [ ]:
df_ext = test_tree.match_dataset(mol_sup=Chem.SDMolSupplier(data_external, removeHs=False))

In [ ]:
calculate_RMSE_tree_vs_truth(df_ext, "truth", "tree_norm2")

In [ ]:
test_df = pd.read_csv(out_folder+"/test_df.csv")

In [ ]:
df_test = test_tree.match_dataset_with_indices(mol_sup=Chem.SDMolSupplier(sdf_suply, removeHs=False), indices=test_df.mol_index.unique().tolist())

In [ ]:
#calculate_RMSE_tree_vs_truth(df_test, "truth", "tree_norm2")

## prune tune

In [ ]:
test_tree = tree()
test_tree.from_folder(out_folder+"/tree")

In [ ]:
test_tree.root.children

In [ ]:
np.log10(0.02)

In [ ]:
thresholds = np.logspace(-4, -1.7, num=32)

In [ ]:
tree_depth=[]

tree_rmse=[]
tree_r2=[]

tree_rmse_solv=[]
tree_r2_solv=[]

tree_rmse_ext=[]
tree_r2_ext=[]

tree_time=[]


for i, threshold in enumerate(thresholds):
    print(i)
    for child in test_tree.root.children:
        child.prune(threshold=threshold)
    test_tree.update_tree_length()
    tree_depth.append(sum(test_tree.tree_lengths.values()))

    # main test
    start_time = time.process_time()
    df_test = test_tree.match_dataset_with_indices(mol_sup=Chem.SDMolSupplier(sdf_suply, removeHs=False), indices=test_df.mol_index.unique().tolist())
    tree_time.append(time.process_time() - start_time)
    tree_rmse.append(calculate_RMSE_tree_vs_truth(df_test, "tree_norm2", "truth"))
    tree_r2.append(calculate_r2_tree_vs_truth(df_test, "tree_norm2", "truth"))

    # solv test
    df_solv = test_tree.match_dataset(mol_sup=Chem.SDMolSupplier(data_solv, removeHs=False))
    tree_rmse_solv.append(calculate_RMSE_tree_vs_truth(df_solv, "tree_norm2", "truth"))
    tree_r2_solv.append(calculate_r2_tree_vs_truth(df_solv, "tree_norm2", "truth"))

    # ext test
    df_ext = test_tree.match_dataset(mol_sup=Chem.SDMolSupplier(data_external, removeHs=False))
    tree_rmse_ext.append(calculate_RMSE_tree_vs_truth(df_ext, "tree_norm2", "truth"))
    tree_r2_ext.append(calculate_r2_tree_vs_truth(df_ext, "tree_norm2", "truth"))


In [ ]:
df_pruning = pd.DataFrame({"threshold": thresholds, "tree_depth": tree_depth, "tree_rmse": tree_rmse, "tree_r2": tree_r2, "tree_rmse_solv": tree_rmse_solv, "tree_r2_solv": tree_r2_solv, "tree_rmse_ext": tree_rmse_ext, "tree_r2_ext": tree_r2_ext, "tree_time": tree_time})

In [ ]:
df_pruning.head(2)

In [ ]:
df_pruning.plot.line(x="threshold", y=["tree_rmse", "tree_rmse_solv", "tree_rmse_ext"])

In [ ]:
plt.figure()
ax = df_pruning.plot.line(x="threshold", y=["tree_rmse", "tree_rmse_solv", "tree_rmse_ext", "tree_depth"], secondary_y=["tree_depth"])
ax.set_xlim([0.001, 0.04])
ax.set_ylabel("RMSE")
ax.right_ax.set_ylabel("Tree Size")

In [ ]:
df_pruning.plot.line(x="threshold", y=["tree_r2", "tree_r2_solv", "tree_r2_ext"])

In [ ]:
ax = df_pruning.plot.line(x="threshold", y="tree_time")
ax.set_yscale("log")

In [ ]:
df_pruning["score"] = df_pruning["tree_time"] * df_pruning["tree_rmse"] * df_pruning["tree_r2"]

In [ ]:
df_pruning.sort_values(by="score", ascending=True).head(10)